# Reasoning

It is an attribute of human beings. The capacity of reasoning allows any person to remember information related to events and make new knowledge about things that we have not lived yet.

This capacity allows us to classify objects and situations, through this we can recognize them and make decisions about them. For example, if someone shows us a computer we can identify it through its features, we can do this with things that we knew previously, for example:

+ It has a mouse
+ It has a monitor
+ It has a keyboard
+ Etc.

## Expert systems

<img src="https://www.igcseict.info/theory/7_2/expert/files/stacks_image_5738.png" />

Since the first artificial intelligence works have tried to develop computers with reasoning capacities like humans. The first systems were called knowledge-based system (KBS) which don't reason but they are specially designed to resolve a specific complex problem through the application of logic rules (it is called the knowledge domain).
These solutions have two main components, they are:

+ The Knowledge Base

Its purpose is to save the information about the domain to make conclusions about the inputs. This is done in the form of _facts_.

+ The Inference Engine

It is the reasoning process of the solution which searches information in the knowledge engine and explores relationships between them them to get a coherent conclusion related to the problem approached.
In a __Expert System__ the _inference engine_ is based in rules and a base of facts or memory of works. 

A fact has information about a domain, and it allows us to associate facts with other new facts via _rules_, an example rule that creates a fact from e previous one is:

FactAntecedent(a) $\to$ FactConsequence(b) 

In the inference engine we can find two search strategies, the most used is called _forward chaining_ and another is named _backward chaining_. In the forward chaining, the expert system makes inferences according to the antecedents of the rules.

For an implementation of forward chaining in Python, we can use the library PyKnow.

## PyKnow

This library allows us to create forward chaining systems that use facts and rules. This implementation of expert systems is based in CLIPS (http://www.clipsrules.net/) which was originally developed by NASA based on the C language.

Let's install the library by running the following line inside a cell in a notebook:

```
!pip install pyknow
```


In [2]:
!pip install https://github.com/buguroo/pyknow/archive/v1.7.0.zip

  Using cached https://github.com/buguroo/pyknow/archive/v1.7.0.zip
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34143 sha256=542c698b39653f927c0c9cfaaf6a4720b7d5970ec9eac5457d2568886c5bf907
  Stored in directory: /private/var/folders/9v/6vrs_kr549z8v97dbfdfmbyr0000gn/T/pip-ephem-wheel-cache-xsa9adau/wheels/14/25/74/76fb8269ad8549713ffa055c0a26d5723f13e626896fb9a1ca
Successfully built pyknow


Now we can import the names in the module:

In [3]:
from pyknow import *

Some definitions of names in the module are:

__Fact__ is a basic unit of PyKnow which are used by the system to reason about the problem. 

__Rule__ is a callable, decorated with _Rule_. Rules have two components, LHS (left-hand-side) and RHS (right-hand-side).

+ The LHS describes (using patterns) the conditions on which the rule _n_ should be executed.
+ The RHS is the set of actions to perform when the rule is fired

```
@Rule(<pattern_1>,
<pattern_2>,
...
<pattern_n>)
def my_rule():
  pass
```

The patterns are applied with AND by default.


For a Fact to match a Pattern, all pattern restrictions must be True when the Fact is evaluated against it. 

In [4]:
f = Fact(a=1, b=2, k=3) # order is arbitraty
print(f['a'])

1


We can make a Fact without keys (only values), and Fact will create a numeric index for each value

In [5]:
f = Fact('x', 'y', 'z')

print(f[1])

y


Variables can also be optionally named.

In [8]:
f = Fact('x', 'y', 'z', a=1, b=2)

print(f[0])
print(f['a'])

x
1


We can subclass Fact to express different kinds of data or extend it with our own properties and functions.

In [9]:
class Alert(Fact):
    pass
class Status(Fact):
    pass

f1 = Alert(color='red')
f2 = Status(state='critical')

print(f1['color'])
print(f2['state'])

red
critical


Most of the time expert systems needs a set of facts to be present for the system to work. This is the purporse of the DefFacts decorator. So, all the DefFacts inside a KnowledgeEngine will be called every time the reset method is executed. 

In [13]:
@DefFacts()
def needed_data():
    yield Fact(best_color='red')
    yield Fact(best_body='medium')
    yield Fact(best_sweetness='dry')

### Rules

Here are some examples of how rules are created in PyKnow.

In [14]:
class MyFact(Fact):
    pass

@Rule(MyFact()) # This is the LHS
def matchWithEveryMYFact():
    # This is the RHS
    pass

For example, let's apply the next rule, math with every Fact which:
+ f[0] == 'animal'
+ f['family'] == 'felinae'

In [10]:
class MyFact(Fact):
    pass

class Animal(Fact):
    pass

@Rule(Animal(family='felinae'))
def match_with_cats():
    print("Meow!")

Another example, the user is a priviliged one and we are not dropping privileges.

In [19]:
class User(Fact):
    pass

@Rule(User('admin')|User('root'))
def the_user_has_power():
    enable_superpowers()

### Conditional Elements (CE)

These help us compose patterns in order to form complex rules.

__AND__

.This creates a composed pattern containing all Facts passed as arguments. All of the passed patterns must match for the composed pattern to match.

In [0]:
# Match if two facts are declared, one martching Fact(1) and other matching Fact(2)
@Rule(
    AND(
        Fact(1),
        Fact(2)
    )
)
def _():
    pass

__OR__

It creates a composed pattern in which any of the given pattern will make the rule match

In [0]:
# Math if a fact matching Fact(1) exists and/or a fact matching Fact(2)
@Rule(OR(Fact(1), Fact(2)))
def _():
    pass

__NOT__

This elements matches if the given pattern does not match with any fact or combination of facts. Therefore this element matches the absence of the given pattern

In [0]:

# Match if not fact with Fact(1)
@Rule(NOT(Fact(1)))
def _():
    pass

__TEST__

It checks the reveived callable against the current binded values. If the execution returns True the evaluation will continue and stops otherwise.

In [11]:
# Match for all numbers a, b, c where a > b > c
class Number(Fact):
    pass

@Rule(
    Number(MATCH.a),
    Number(MATCH.b),
    TEST(lambda a, b: a > b),
    Number(MATCH.c),
    TEST(lambda b, c: b > c)
)
def _(a, b, c):
    print(a, b, c)

__EXISTS__

EXISTS receives a pattern and matches if one or more facts matches this pattern. This will match only once while one ore more matching facts exists and will stop matching when there is no matching facts.

In [0]:
# Match once when one or more Color exists
class Color(Fact):
    pass

@Rule(EXISTS(Color()))
def _():
    pass

## Field Constraints: (FC)

__L (Literal Field Constraint)__

It allows the program to perform an exact match with the given value. The matching is donde using the equality operator ==.

In [0]:
# Match if the first element is exactly 3
@Rule(Fact(L(3)))
def _():
    pass

__W (Wildcard Field Constraint)__

This element matches with any value.

In [0]:
# match if some fact is declared with the key mykey
@Rule(Fact(mykey=W()))
def _():
      pass

__P (Predicate Field Constraint):__

The match of this element is the result of applying the given callable to the fact-extracted value. If the callable returns True the FC will match, in other case the FC will not match. 

In [13]:
# match is some fact is declared whose first parameter is an instance of int
@Rule(
    Fact(
        P(lambda y: isinstance(y, int))
    )
)
def _():
    pass

## Composing FCs: &, | and ~

__ANDFC() - &__

The composed FC matches if all the given FC match.


In [0]:
# Match if key x of Point is a value between 0 and 255
@Rule(
    Fact(
        x=(P(lambda x: x >= 0) & P(lambda x: x <= 255))
    )
)
def _():
    pass

__ORFC() - |__

The composed FC matches if any of the given FC matches.

In [0]:
@Rule(
    Fact(
        name=(L('Alice') | L('Bob'))
    )
)
def _():
    pass

__NOTFC() - ~__

This composed FC negates the given FC, reversing the logic FC. 

In [0]:
# Match if name is not Charlie
@Rule(
    Fact(
        name=~L('Charlie')
    )
)
def _():
    pass

## Variable Binding: The << Operator

Any pattern and some FC's can be binded to a name using the << operator.

In [0]:
# The first value of the matching fact will be blinded to the name value and passed to the function when fired
@Rule(
    Fact(
        'value' << W()
    )
)
def _(value):
    pass

### MATCH Object

This allows us to generate more readable name bindings.

In [0]:
@Rule(Fact(MATCH.arbitrario))
def _(arbitrario):
    pass

The last example is exactly the same as:

```
@Rule(Fact('myvalue' << W()))
def _(myvalue):
    pass
```

### AS Object

The AS object like the MATCH object is syntatic sugar for generating bindable names. In this case any attribute requested to the AS object will return a string with the same name.

In [8]:
@Rule(AS.myfact << Fact(W()))
def _(myfact):
    print(myfact[0])

The last example is exactly the same as:

```
@Rule("myfact" << Fact(W()))
def _(myfact):
    pass
```

## Examples

### Trafic Lights

In the next example we can implement our first hello world of expert systems in Python, look how we made the rules and how we defined their results. 

In [15]:
from random import choice
from pyknow import *

class Light(Fact):
    """Info about the traffic light."""
    pass


class RobotCrossStreet(KnowledgeEngine):
    @Rule(Light(color='green'))
    def green_light(self):
        print("Walk")

    @Rule(Light(color='red'))
    def red_light(self):
        print("Don't walk")

    @Rule(AS.xx << Light(
            color=L('yellow') | L('blinking-yellow')
        )
    )
    def cautious(self, xx):
        print("Be cautious because light is", xx["color"])

The knowledgeEngine is the place where all the magic happens, the first step is to make a subclass of it and use Rule to decorate its methods. The next step is to instantiate it and run it. 

This is the usual process to execute a KnowledgeEngine.

+ The class must be instantiated, of course.
+ The reset method must be called:
    + This declares the special fact InitialFact. Necessary for some rules to work properly.
    + Declare all facts yielded by the methods decorated with @DefFacts.
+ The run method must be called. This starts the cycle of execution.

In [24]:
engine = RobotCrossStreet()
engine.reset()
engine.declare(Light(color=choice(['green', 'yellow', 'blinking-yellow', 'red'])))
engine.run()

Be cautious because light is blinking-yellow


### Chat

In [29]:
from random import choice
from pyknow import *

class Greetings(KnowledgeEngine):
    # Most of the time expert systems needs a set of facts to be present 
    # for the system to work. This is the purpose of the DefFacts decorator
    @DefFacts()
    def _initial_action(self):
        yield Fact(action="greet")

    @Rule(Fact(action='greet'),
          NOT(Fact(name=W())))
    def ask_name(self):
        self.declare(Fact(name=input("What's your name? ")))

    @Rule(Fact(action='greet'),
            Fact(name=W()),
          NOT(Fact(location=W())))
    def ask_location(self):
        self.declare(Fact(location=input("Where are you? ")))

    @Rule(Fact(action='greet'),
          Fact(name=MATCH.name),
          Fact(location=MATCH.location))
    def greet(self, name, location):
        print("Hi %s! How is the weather in %s?" % (name, location))

engine = Greetings()
engine.reset()  # Prepare the engine for the execution.
engine.run()  # Run it!

Hi Felipe! How is the weather in Cali?


## References

+ https://media.readthedocs.org/pdf/pyknow/latest/pyknow.pdf
+ https://es.slideshare.net/ahmadhussein45/expert-system-with-python-2